#### Example

```
{"query": {"user_id": ["1"], "bucketized_user_age": [18.0], "raw_user_age": [24.0], "timestamp": [0.012986151007588376], "user_gender": ["Male"], "user_occupation_label": [18], "user_occupation_text": ["technician"], "user_zip_code": ["85711"]}, "candidate": {"movie_id": ["1"], "movie_title": ["Toy Story"], "year": [1995], "movie_genres": ["2", "3", "4"]}}
```

In [21]:
?client.list_rows

Signature:
client.list_rows(
    table: Union[google.cloud.bigquery.table.Table, google.cloud.bigquery.table.TableListItem, google.cloud.bigquery.table.TableReference, str],
    selected_fields: Sequence[google.cloud.bigquery.schema.SchemaField] = None,
    max_results: int = None,
    page_token: str = None,
    start_index: int = None,
    page_size: int = None,
    retry: google.api_core.retry.Retry = <google.api_core.retry.Retry object at 0x7f6edcda2450>,
    timeout: Union[float, NoneType] = None,
) -> google.cloud.bigquery.table.RowIterator
Docstring:
List the rows of the table.

See
https://cloud.google.com/bigquery/docs/reference/rest/v2/tabledata/list

.. note::

   This method assumes that the provided schema is up-to-date with the
   schema as defined on the back-end: if the two schemas are not
   identical, the values returned may be incomplete. To ensure that the
   local copy of the schema is up-to-date, call ``client.get_table``.

Args:
    table (Union[                 

In [ ]:
import json


from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

query = "select * from `hybrid-vertex.spotify_train_3.train_flatten`"
# data = client.query(query)
# data.allow_large_results = True
data = client.list_rows("hybrid-vertex.spotify_train_3.train_flatten")

query_fields = [
    'name',
    'collaborative',
    'description',
    'duration_ms_playlist',
    'artist_name_seed_track',
    'artist_uri_seed_track',
    'track_name_seed_track',
    'track_uri_seed_track',
    'album_name_seed_track',
    'album_uri_seed_track',
    'duration_seed_track',
    'duration_ms_seed_pl',
    'n_songs',
    'num_artists',
    'num_albums',
    'artist_name_seed_pl',
    'track_uri_seed_pl',
    'track_name_seed_pl',
    'duration_ms_seed_songs_pl',
    'album_name_seed_pl',
    'artist_pop_seed_pl',
    'artists_followers_seed_pl',
    'track_pop_seed_pl',
]

candidate_fields = [
    'artist_name_seed',
    'track_uri_seed',
    'artist_uri_seed',
    'track_name_seed',
    'album_uri_seed',
    'duration_ms_seed',
    'album_name_seed',
    'track_pop_seed',
    'artist_pop_seed',
    'artist_genres_seed',
    'artist_followers_seed',
]


#iterate over the bq rows and update corresponding data jsonl data
with open('train_data.jsonl', 'w') as file:
    for row in data:
        line = {"query" : {},
               "candidate" : {}}
        for field in query_fields:
            line["query"].update({field: row[field]})
        for field in candidate_fields:
            line["candidate"].update({field: row[field]})
        file.write(json.dumps(line) + ",\n")
        
file.close()

In [ ]:
a